# NSU Distributional Semantics 2019 Course. Seminar 4¶

In this seminar, we will see how to train a Word2Vec model and then how to build a sentiment analyser for tweets.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing

In [2]:
import gzip
import gensim 
import logging
import requests

In [3]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [4]:
#data_url = 'https://www.gutenberg.org/files/1703/1703-0.txt'
#text = requests.get(data_url).text.split('.')

In [5]:
!ls ../input/
data = pd.read_csv('../input/nips-2015-papers/Papers.csv')

nips-2015-papers  twitter-emotion


In [6]:
#Create a function which return lines preprocessed thanks to gensim.utils.simple_preprocess

def read_line(file):
  ratio = len(file)/100
  for i,line in enumerate(file):
    if (i % ratio == 0): logging.info("read {0} lines".format(i))
    yield gensim.utils.simple_preprocess(line)

In [7]:
documents = list (read_line(data['PaperText']))
print(documents[1])

2019-03-17 16:48:26,870 : INFO : read 0 lines


['learning', 'with', 'symmetric', 'label', 'noise', 'the', 'importance', 'of', 'being', 'unhinged', 'brendan', 'van', 'rooyen', 'aditya', 'krishna', 'menon', 'the', 'australian', 'national', 'university', 'robert', 'williamson', 'national', 'ict', 'australia', 'brendan', 'vanrooyen', 'aditya', 'menon', 'bob', 'williamson', 'nicta', 'com', 'au', 'abstract', 'convex', 'potential', 'minimisation', 'is', 'the', 'de', 'facto', 'approach', 'to', 'binary', 'classification', 'however', 'long', 'and', 'servedio', 'proved', 'that', 'under', 'symmetric', 'label', 'noise', 'sln', 'minimisation', 'of', 'any', 'convex', 'potential', 'over', 'linear', 'function', 'class', 'can', 'result', 'in', 'classification', 'performance', 'equivalent', 'to', 'random', 'guessing', 'this', 'ostensibly', 'shows', 'that', 'convex', 'losses', 'are', 'not', 'sln', 'robust', 'in', 'this', 'paper', 'we', 'propose', 'convex', 'classification', 'calibrated', 'loss', 'and', 'prove', 'that', 'it', 'is', 'sln', 'robust', 'th

In [8]:
model = gensim.models.Word2Vec(
        documents,
        size=200,
        window=10,
        min_count=2,
        workers=10)
model.train(documents, total_examples=len(documents), epochs=10)



2019-03-17 16:48:29,840 : INFO : collecting all words and their counts
2019-03-17 16:48:29,842 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-03-17 16:48:30,238 : INFO : collected 37245 word types from a corpus of 1835093 raw words and 403 sentences
2019-03-17 16:48:30,240 : INFO : Loading a fresh vocabulary
2019-03-17 16:48:30,379 : INFO : effective_min_count=2 retains 23104 unique words (62% of original 37245, drops 14141)
2019-03-17 16:48:30,380 : INFO : effective_min_count=2 leaves 1820952 word corpus (99% of original 1835093, drops 14141)
2019-03-17 16:48:30,473 : INFO : deleting the raw counts dictionary of 37245 items
2019-03-17 16:48:30,477 : INFO : sample=0.001 downsamples 30 most-common words
2019-03-17 16:48:30,479 : INFO : downsampling leaves estimated 1434722 word corpus (78.8% of prior 1820952)
2019-03-17 16:48:30,593 : INFO : estimated required memory for 23104 words and 200 dimensions: 48518400 bytes
2019-03-17 16:48:30,594 : INFO : rese

(14347087, 18350930)

In [9]:
w1 = "development"
model.wv.most_similar (positive=w1)

2019-03-17 16:49:05,268 : INFO : precomputing L2-norms of word weight vectors


[('extensive', 0.5657904744148254),
 ('understanding', 0.5527400374412537),
 ('developments', 0.5486680269241333),
 ('opens', 0.5451460480690002),
 ('progress', 0.5385216474533081),
 ('scientists', 0.5344961285591125),
 ('investigation', 0.5316309928894043),
 ('breakthroughs', 0.5298246145248413),
 ('studies', 0.5286712646484375),
 ('developing', 0.5261154174804688)]

In [10]:
w1 = "man"
model.wv.most_similar (positive=w1)

[('perona', 0.7742531895637512),
 ('fish', 0.7723714113235474),
 ('horse', 0.7706104516983032),
 ('vondrick', 0.769294023513794),
 ('welinder', 0.7668565511703491),
 ('frisbee', 0.7664816379547119),
 ('comic', 0.7657363414764404),
 ('basal', 0.765304684638977),
 ('julius', 0.7632596492767334),
 ('sea', 0.7630102634429932)]

In [11]:
w1 = "woman"
model.wv.most_similar (positive=w1)

[('costume', 0.959667444229126),
 ('girl', 0.8462927341461182),
 ('julius', 0.7994595170021057),
 ('shaoqingren', 0.7970902323722839),
 ('scgb', 0.7940157055854797),
 ('faster_rcnn', 0.7921811938285828),
 ('sea', 0.7911180853843689),
 ('americans', 0.7891307473182678),
 ('chart', 0.7882951498031616),
 ('omj', 0.7880900502204895)]

In [12]:
w1 = "dog"
model.wv.most_similar (positive=w1)

[('cat', 0.9085661768913269),
 ('cow', 0.8652568459510803),
 ('horse', 0.8493620157241821),
 ('firehydrant', 0.8407943844795227),
 ('bottle', 0.8394757509231567),
 ('boat', 0.8227623701095581),
 ('bananas', 0.8137739896774292),
 ('sheep', 0.80582195520401),
 ('sofa', 0.8014547824859619),
 ('hydrant', 0.7759835124015808)]

In [13]:
w1 = "god"
model.wv.most_similar (positive=w1)

[('christians', 0.9725555181503296),
 ('jesus', 0.8978714942932129),
 ('religion', 0.6373331546783447),
 ('φki', 0.6171691417694092),
 ('αpki', 0.6130654811859131),
 ('factory', 0.6121131181716919),
 ('χm', 0.6107220649719238),
 ('φkj', 0.6015666723251343),
 ('boy', 0.5992591381072998),
 ('bowl', 0.5966882109642029)]

Most of the results here are non sense, this is due to the corpora used for the training. Try this again with another training set.

# Sentiment analysing based on tweets

This project is inspired by [this blog](https://ahmedbesbes.com/sentiment-analysis-on-twitter-using-word2vec-and-keras.html) where based on a certain collection of tweets, we try to see whether or not their meaning is positive or negative.

## Data

The dataset is a dataset from Kaggle with a certain amount of tweets available.

In [14]:
data = pd.read_csv('../input/twitter-emotion/train.csv', engine='python')

In [15]:
data.head()

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...


In [16]:
#No need of the first row
data = data[['Sentiment', 'SentimentText']]

In [17]:
data.head()

,Sentiment,SentimentText
0,0,is so sad for my APL frie...
1,0,I missed the New Moon trail...
2,1,omg its already 7:30 :O
3,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,0,i think mi bf is cheating on me!!! ...


## Data cleaning

Before using tweets, we need to clean them, this means mainly to get read of @ # and links.

In [18]:
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

In [19]:
def tokenize(tweet):
    try:
        tweet = str(tweet.lower())
        tokens = tokenizer.tokenize(tweet)
        tokens = [tok for tok in tokens if '#' not in tok]
        tokens = [tok for tok in tokens if 'http' not in tok]
        tokens = [tok for tok in tokens if '@' not in tok]
        return tokens
    except:
        return 'NC'

In [20]:
def postprocess(data, n=1000000):
    data = data.head(n)
    data['tokens'] = data['SentimentText'].progress_map(tokenize)
    data = data[data.tokens != 'NC']
    data.reset_index(inplace=True)
    data.drop('index', inplace=True, axis=1)
    return data

data = postprocess(data)

progress-bar: 100%|██████████| 99989/99989 [00:05<00:00, 18536.18it/s]
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
data.head().tokens

0             [is, so, sad, for, my, apl, friend, ...]
1            [i, missed, the, new, moon, trailer, ...]
2                      [omg, its, already, 7:30, :, o]
3    [.., omgaga, ., im, sooo, im, gunna, cry, ., i...
4    [i, think, mi, bf, is, cheating, on, me, !, !,...
Name: tokens, dtype: object

# Data preparation

All the data are splitted into different sets to setup a training and test phase.

In [22]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(np.array(data.head(100000).tokens),
                                                    np.array(data.head(100000).Sentiment), test_size=0.2)


In [23]:
x_train

array([list(['well', 'said', '...', 'please', 'take', 'him', 'off', 'my', 'hands', '..', 'i', "can't", 'cope', '!', '!', '!', 'set', 'that', 'fox', 'on', 'him']),
       list(["that's", 'all', 'i', 'have', 'to', 'say', 'or', 'show', 'lol']),
       list(['kimora', 'congrats', 'to', 'you', 'and', 'djimon', 'on', 'your', 'new', 'baby', 'boy', '...', 'blessings']),
       ...,
       list(['speed', 'in', 'for', 'joe', 'again', 'check', 'out', 'the', 'article', 'i', 'posted', 'on', 'jnn']),
       list(['mann', '.', 'i', 'wish', 'i', "could've", 'gone', '!']),
       list(['glad', 'you', 'had', 'a', 'great', 'time', ',', 'but', "i'm", 'sad', 'too', 'it', 'was', 'just', 'amazing', 'last', 'night', '...', "there's", 'nothing', 'like', 'a', 'toronto', 'crowd', '!'])],
      dtype=object)

In [24]:
import gensim
from gensim.models.word2vec import Word2Vec
LabeledSentence = gensim.models.doc2vec.LabeledSentence

def labelizeTweets(tweets, label_type):
    labelized = []
    for i,v in tqdm(enumerate(tweets)):
        label = '%s_%s'%(label_type,i)
        labelized.append(LabeledSentence(v, [label]))
    return labelized

x_train = labelizeTweets(x_train, 'TRAIN')
x_test = labelizeTweets(x_test, 'TEST')

0it [00:00, ?it/s]/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  if __name__ == '__main__':
79991it [00:00, 132203.34it/s]
19998it [00:00, 55745.83it/s]


In [25]:
x_train[10]

LabeledSentence(words=['thunder', 'and', 'lightning', '.', 'one', 'thing', 'i', 'am', 'scared', 'of', '.', 'even', 'bear', 'is', 'scared'], tags=['TRAIN_10'])

In [26]:
model = Word2Vec(size=200, min_count=5)
model.build_vocab([x.words for x in tqdm(x_train)])
model.train([x.words for x in tqdm(x_train)], total_examples=len([x.words for x in tqdm(x_train)]),epochs=10)

100%|██████████| 79991/79991 [00:00<00:00, 1501409.96it/s]
2019-03-17 16:49:13,955 : INFO : collecting all words and their counts
2019-03-17 16:49:13,956 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-03-17 16:49:13,998 : INFO : PROGRESS: at sentence #10000, processed 144097 words, keeping 13539 word types
2019-03-17 16:49:14,042 : INFO : PROGRESS: at sentence #20000, processed 291532 words, keeping 21117 word types
2019-03-17 16:49:14,090 : INFO : PROGRESS: at sentence #30000, processed 436064 words, keeping 27085 word types
2019-03-17 16:49:14,140 : INFO : PROGRESS: at sentence #40000, processed 580320 words, keeping 32212 word types
2019-03-17 16:49:14,188 : INFO : PROGRESS: at sentence #50000, processed 725958 words, keeping 37004 word types
2019-03-17 16:49:14,235 : INFO : PROGRESS: at sentence #60000, processed 870812 words, keeping 41270 word types
2019-03-17 16:49:14,281 : INFO : PROGRESS: at sentence #70000, processed 1015910 words, keeping 453

(7938218, 11614530)

In [27]:
model.most_similar(['vodka'])
#model.most_similar(positive=['vodka', 'france'], negative=['russia'])
#Paris
w1 = "man"
model.wv.most_similar (positive=w1)
w1 = "woman"
model.wv.most_similar (positive=w1)
w1 = "dog"
model.wv.most_similar (positive=w1)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
2019-03-17 16:49:27,210 : INFO : precomputing L2-norms of word weight vectors


[('cat', 0.7416350841522217),
 ('boyfriend', 0.7170374393463135),
 ('brother', 0.6902833580970764),
 ('kid', 0.6809959411621094),
 ('heart', 0.6758929491043091),
 ('teacher', 0.6732697486877441),
 ('wife', 0.6538675427436829),
 ('kitty', 0.6504315137863159),
 ('daddy', 0.6460946202278137),
 ('boy', 0.6455011963844299)]

Results now make more sense even if it may be surprising; this may be seen as the "tweeter" effect: tweets are short and quite explicite.

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [29]:
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
matrix = vectorizer.fit_transform([x.words for x in x_train])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print ('vocab size :', len(tfidf))

vocab size : 5334


In [30]:
def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += model[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [31]:
from sklearn.preprocessing import scale
train_vecs_w2v = np.concatenate([buildWordVector(z, 200) for z in tqdm(map(lambda x: x.words, x_train))])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([buildWordVector(z, 200) for z in tqdm(map(lambda x: x.words, x_test))])
test_vecs_w2v = scale(test_vecs_w2v)

0it [00:00, ?it/s]/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
79991it [00:13, 5876.98it/s]
0it [00:00, ?it/s]/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
19998it [00:03, 5629.70it/s]


In [32]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

modelF = Sequential()
modelF.add(Dense(32, activation='relu', input_dim=200))
modelF.add(Dense(1, activation='sigmoid'))
modelF.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

modelF.fit(train_vecs_w2v, y_train, epochs=9, batch_size=32, verbose=2)

Using TensorFlow backend.


Epoch 1/9
 - 4s - loss: 0.5348 - acc: 0.7301
Epoch 2/9
 - 3s - loss: 0.5132 - acc: 0.7450
Epoch 3/9
 - 3s - loss: 0.5060 - acc: 0.7513
Epoch 4/9
 - 3s - loss: 0.5023 - acc: 0.7535
Epoch 5/9
 - 3s - loss: 0.4989 - acc: 0.7563
Epoch 6/9
 - 3s - loss: 0.4962 - acc: 0.7580
Epoch 7/9
 - 3s - loss: 0.4945 - acc: 0.7582
Epoch 8/9
 - 3s - loss: 0.4923 - acc: 0.7598
Epoch 9/9
 - 3s - loss: 0.4909 - acc: 0.7611


In [33]:
score = modelF.evaluate(test_vecs_w2v, y_test, batch_size=128, verbose=2)
print (score[1])
print(x_test[1], y_test[1])



0.7512751274948681
LabeledSentence(['my', 'night', 'was', 'completely', 'devoid', 'of', 'lips', '.', 'at', 'least', 'lips', 'intended', 'for', 'me', '.', 'you', 'were', 'barely', 'alive', 'the', 'last', 'time', 'they', 'were', 'haha'], ['TEST_1']) 0
